#### Vector stores and retrievers
This video tutorial will familiarize you with LangChain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data — from (vector) databases and other sources — for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.
We will cover

* Documents
* Vector stores
* Retrievers

### Documents
LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content;
- metadata: a dict containing arbitrary metadata. The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

Lets generate some sample documents

In [1]:
from langchain_core.documents import Document

doc_data = [
    {
        "content": "Real Madrid has won 15 UCLs Title till today which is the first club in history to do so!!",
        "metadata": {"source": "football-world", "topic": "ucl_records"}
    },
    {
        "content": "Real Madrid was founded in 1902 and plays their home matches at Santiago Bernabéu Stadium.",
        "metadata": {"source": "football-history", "topic": "club_info"}
    },
    {
        "content": "Cristiano Ronaldo scored 450 goals for Real Madrid between 2009 and 2018.",
        "metadata": {"source": "player-stats", "topic": "goal_records"}
    }
]

documents = [Document(page_content=item["content"], metadata=item["metadata"]) for item in doc_data]

In [2]:
documents

[Document(metadata={'source': 'football-world', 'topic': 'ucl_records'}, page_content='Real Madrid has won 15 UCLs Title till today which is the first club in history to do so!!'),
 Document(metadata={'source': 'football-history', 'topic': 'club_info'}, page_content='Real Madrid was founded in 1902 and plays their home matches at Santiago Bernabéu Stadium.'),
 Document(metadata={'source': 'player-stats', 'topic': 'goal_records'}, page_content='Cristiano Ronaldo scored 450 goals for Real Madrid between 2009 and 2018.')]

In [3]:
import os 
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm=ChatGroq(model="openai/gpt-oss-20b",groq_api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021D91E4F5E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021D91EDFF70>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [5]:
## VEctorStores

from langchain_chroma import Chroma

## All of the documents needs to be converted into embeddings using the embedding technique that we have loaded and store them into the vectordb
vectorstore = Chroma.from_documents(documents, embedding=embeddings)
vectorstore

In [7]:
vectorstore.similarity_search("Cristiano Ronaldo")

[Document(id='f7233784-e07f-4ee9-b4b6-528166f628db', metadata={'topic': 'goal_records', 'source': 'player-stats'}, page_content='Cristiano Ronaldo scored 450 goals for Real Madrid between 2009 and 2018.'),
 Document(id='dbda6a14-2651-44f5-952a-99cf9f984be5', metadata={'topic': 'ucl_records', 'source': 'football-world'}, page_content='Real Madrid has won 15 UCLs Title till today which is the first club in history to do so!!'),
 Document(id='846fc83f-5ed8-458e-ba2c-532efbb7fb2c', metadata={'source': 'football-history', 'topic': 'club_info'}, page_content='Real Madrid was founded in 1902 and plays their home matches at Santiago Bernabéu Stadium.')]

In [8]:
# Async Query

await vectorstore.asimilarity_search("Cristiano Ronaldo")

[Document(id='f7233784-e07f-4ee9-b4b6-528166f628db', metadata={'source': 'player-stats', 'topic': 'goal_records'}, page_content='Cristiano Ronaldo scored 450 goals for Real Madrid between 2009 and 2018.'),
 Document(id='dbda6a14-2651-44f5-952a-99cf9f984be5', metadata={'topic': 'ucl_records', 'source': 'football-world'}, page_content='Real Madrid has won 15 UCLs Title till today which is the first club in history to do so!!'),
 Document(id='846fc83f-5ed8-458e-ba2c-532efbb7fb2c', metadata={'topic': 'club_info', 'source': 'football-history'}, page_content='Real Madrid was founded in 1902 and plays their home matches at Santiago Bernabéu Stadium.')]

In [9]:
vectorstore.similarity_search_with_score("Cristiano Ronaldo")

[(Document(id='f7233784-e07f-4ee9-b4b6-528166f628db', metadata={'source': 'player-stats', 'topic': 'goal_records'}, page_content='Cristiano Ronaldo scored 450 goals for Real Madrid between 2009 and 2018.'),
  0.8350873589515686),
 (Document(id='dbda6a14-2651-44f5-952a-99cf9f984be5', metadata={'topic': 'ucl_records', 'source': 'football-world'}, page_content='Real Madrid has won 15 UCLs Title till today which is the first club in history to do so!!'),
  1.4925806522369385),
 (Document(id='846fc83f-5ed8-458e-ba2c-532efbb7fb2c', metadata={'source': 'football-history', 'topic': 'club_info'}, page_content='Real Madrid was founded in 1902 and plays their home matches at Santiago Bernabéu Stadium.'),
  1.6212519407272339)]

### Retrivers

LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

In [10]:
from typing import List


from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriver = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriver.batch(["Cristiano Ronaldo","UCL"])

[[Document(id='f7233784-e07f-4ee9-b4b6-528166f628db', metadata={'topic': 'goal_records', 'source': 'player-stats'}, page_content='Cristiano Ronaldo scored 450 goals for Real Madrid between 2009 and 2018.')],
 [Document(id='dbda6a14-2651-44f5-952a-99cf9f984be5', metadata={'source': 'football-world', 'topic': 'ucl_records'}, page_content='Real Madrid has won 15 UCLs Title till today which is the first club in history to do so!!')]]

VectorStores implement an as_retriver method that will generate a Retriver, specifically a VectorStoreRetriver. These retrivers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to paramerterize them . For instance, we can replicate the above with the following.

In [11]:
retriver = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":1}
)

retriver.batch(["Cristiano Ronaldo","UCL"])

[[Document(id='f7233784-e07f-4ee9-b4b6-528166f628db', metadata={'topic': 'goal_records', 'source': 'player-stats'}, page_content='Cristiano Ronaldo scored 450 goals for Real Madrid between 2009 and 2018.')],
 [Document(id='dbda6a14-2651-44f5-952a-99cf9f984be5', metadata={'source': 'football-world', 'topic': 'ucl_records'}, page_content='Real Madrid has won 15 UCLs Title till today which is the first club in history to do so!!')]]

In [ ]:
### Integrating retriver with the chain (Basic RAG)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer the following question using the provided content only.

{question}

Context: {context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

# retriver basically has the information all from our vector store 
rag_chain = {"context": retriver, "question": RunnablePassthrough()}|prompt|llm

response = rag_chain.invoke("Tell me about Real Madrid and Cristiano Ronaldo connection")

print(response.content)

Cristiano Ronaldo’s connection to Real Madrid is highlighted by the fact that he scored **450 goals for the club between 2009 and 2018**. This statistic underscores his significant impact and legacy at Real Madrid during that period.
